In [ ]:
%matplotlib inline
import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import numpy as np
from scipy.interpolate import interp1d
from scipy.optimize import brentq
from scipy.signal import tukey
import pandas as pd
import clemb
from clemb.forward_model import forward_model, fullness, esol, rk4, es
import IPython.core.debugger
dbg = IPython.core.debugger.Pdb()


In [ ]:
def error(y1, y2, t1, t2):
    f1 = interp1d(t1, y1)
    err = y2 - f1(t2)
    return err

In [ ]:
if True:
    c = clemb.Clemb(clemb.LakeDataFITS(), clemb.WindDataCSV(), start='2018-02-01 ', end='2018-06-06')

In [ ]:
if True:
    rp = c.run_backward()

In [ ]:
pwr = rp['pwr']*0.0864
datetime = pd.to_datetime(rp['dates'].data)
melt = rp['fmelt']
orig_steam = rp['steam']
orig_mevap = rp['evfl']
ll = rp['llvl']
t = rp['t']

ndays = (datetime[-1] - datetime[0]).days
dt = 1.
nsteps = int(ndays/float(dt))
time = np.linspace(0,ndays,nsteps+1)
T0 = t.data[0]

y = np.zeros((nsteps+1, 3))
a,vol = fullness(ll.data)
density = 1.003 - 0.00033 * T0
M0 = vol[0]*density
y[0] = [T0, M0, 0]
steams = np.zeros(nsteps+1)
mevaps = np.zeros(nsteps+1)
esols = np.zeros(nsteps+1)
for i in range(nsteps):
    solar = esol(dt, a[i], datetime[i].month)
    esols[i] = solar
    Mout = 0.
    enthalpy = 6.0
    windspeed = 4.5
    y_new, steam, mevap = forward_model(y[i], dt, a[i], vol[i], pwr.values[i], 
                                 melt.values[i], Mout, solar, enthalpy, windspeed)
    steams[i] = steam
    mevaps[i] = mevap
    y[i+1] = y_new
    
mpl.rcParams['figure.subplot.hspace'] = 0.5
plt.figure(figsize=(10,8))
ax1 = plt.subplot(4,2,1)
ax1.plot(time, pwr.values[:]/0.0864, ls='--')
ax1.set_title('Heat input rate')

ax2 = plt.subplot(4,2,3)
ax2.plot(time, melt.values[:], ls='--')
ax2.set_title('Meltwater inflow')

ax3 = plt.subplot(4,2,5)
ax3.plot(time, mevaps)
ax3.plot(time, orig_mevap, ls='--')
ax3.set_title('Evaporation')

ax4 = plt.subplot(4,2,2)
ax4.plot(time, y[:,0])
ax4.plot(time, t.values[:], ls='--')
ax4.set_title('Lake temperature')

ax5 = plt.subplot(4,2,4)
ax5.plot(time, y[:,0] - t.values[:])
ax5.set_title('Temperature difference')

ax6 = plt.subplot(4,2,6)
ax6.plot(time, steams)
ax6.plot(time, orig_steam, ls='--')
ax6.set_title('Steam')

ax7 = plt.subplot(4,2,7)
ax7.plot(time, y[:,1])
_, vol = fullness(ll.values[:])
rho = 1.003 - 0.00033 * t.values[:]
mass = vol*rho
ax7.plot(time, mass, ls='--')
ax7.set_title('Lake mass')

ax8 = plt.subplot(4,2,8)
ax8.plot(time, y[:,1]- mass)
ax8.set_title('Mass difference')


### Synthetic input

In [ ]:
# matplotlib.rcParams['axes.prop_cycle']
cl1 = '#1f77b4'
cl2 = '#ff7f0e'
cl3 = '#2ca02c'

In [ ]:
def outflow(level, area=0.2):
    """
    Compute outflow based on bernoulli equation.
    Returns outflow in km^3/day.
    """
    g = 9.81 # m/s^2
    h = level - 45.31
    if h <= 0.:
        return 0.
    v = np.sqrt(2*g*h)
    vol = area * v # ouflow volume in m^3/s
    vol *= 1e-3 * 86400 # convert to km^3/day
    return vol

In [ ]:
def volume(mass, temp):
    """
    Compute volume based on mass and temperature.
    Return volume in km^3
    """
    rho = 1.003 - 0.00033 * temp
    return mass/rho

In [ ]:
def mass(volume, temp):
    """
    Return mass based on volume and temperature.
    """
    rho = 1.003 - 0.00033 * temp
    return volume * rho

In [ ]:
def level(volume, lakearea):
    """
    Compute lake level based on volume and surface area.
    Return lake level in m.
    """
    return volume * 1e3 / lakearea

In [ ]:
f = 1/15.
nsteps = 101 #31 
tmax = 30.
dates = pd.date_range(start='1/1/2017', end='31/1/2017', periods=nsteps)
dt = (dates[1] - dates[0])/pd.Timedelta('1D')
t = np.linspace(0, tmax, nsteps)
_lambda = .1
qi_sinus = False
qi_tukey = True
if qi_sinus:
    qi = np.sin(2.*np.pi*f*t + np.deg2rad(15))*250.+250.
    #qi *= np.exp(-_lambda*t)
    qi *= 0.0864
    qi *= tukey(nsteps, 0.9)
if qi_tukey:
    tail = int(10/dt)
    qi = np.ones(nsteps-tail)*500.
    qi *= tukey(nsteps-tail, 0.9)
    qi_tmp = np.zeros(nsteps)
    qi_tmp[0:nsteps-tail] = qi
    qi = qi_tmp
    

y = np.zeros((nsteps, 3))
prm = np.zeros((nsteps, 5))
T = 15.
V = 8800
A = 194162
Mc = 10.
H = 6.0
WS = 4.5
X = 2.
M = mass(V, T)
y[0,:] = [T, M, X]
ll = level(V, A)
Mo = outflow(ll)

for i in range(nsteps-1):
    prm[i, 0] = Mc
    prm[i, 1] = Mo
    prm[i, 2] = ll
    dt = (dates[i+1] - dates[i])/pd.Timedelta('1D')
    solar = esol(dt, A, dates[i].month)
    y_new, steam, mevap = forward_model(y[i], dt, A, V, qi[i]*0.0864,
                                        Mc, Mo, solar, H, WS,
                                        method='euler')
    prm[i, 3] = mevap
    prm[i, 4] = solar
    V = volume(y_new[1], y_new[0])
    ll = level(V, A)
    Mo = outflow(ll)
    y[i+1, :] = y_new[:]
prm[i+1, 0] = Mc
prm[i+1, 1] = Mo
prm[i+1, 2] = ll
prm[i+1, 3] = mevap
prm[i+1, 3] = solar

# Prescripe errors
T_err = 0.4*dt
z_err = 0.01*dt
M_err = 2.*dt
X_err = 0.4*dt
A_err = 30.*dt
V_err = 2.0*dt
Mg_err = 50*dt

# Design dataset
syn_data = {}
syn_data['T'] = y[:,0]
#syn_data['T_err'] = np.random.normal(scale=T_err,size=nsteps)
syn_data['T_err'] = np.ones(nsteps)*T_err
syn_data['p_mean'] = 1.003 - 0.00033 * syn_data['T']
syn_data['p_err'] = 0.00033*np.random.normal(scale=T_err,size=nsteps)
syn_data['a'] = np.ones(nsteps)*A
#syn_data['a_err'] = np.random.normal(scale=A_err, size=nsteps)
syn_data['a_err'] = np.ones(nsteps)*A_err
syn_data['M'] = y[:, 1]
#syn_data['M_err'] = np.random.normal(scale=M_err,size=nsteps)
syn_data['M_err'] = np.ones(nsteps)*M_err
syn_data['v'] = volume(syn_data['M'], syn_data['T'])
syn_data['v_err'] = V_err
syn_data['X'] = y[:,2]
#syn_data['X_err'] = np.random.normal(scale=X_err,size=nsteps)
syn_data['X_err'] = np.ones(nsteps)*X_err
syn_data['Mg'] = syn_data['X']/syn_data['M']*1e6
syn_data['Mg_err'] = Mg_err
syn_data['z'] = prm[:,2]
#syn_data['z_err'] = np.random.normal(scale=z_err,size=nsteps)
syn_data['z_err'] = np.ones(nsteps)*z_err
syn_data['W'] = np.ones(nsteps)*4.5
syn_data['H'] = np.ones(nsteps)*6.0
syn_data['dv'] = np.ones(nsteps)*1.0
df = pd.DataFrame(syn_data, index=dates)

if True:
    mpl.rcParams['figure.subplot.hspace'] = 0.5
    fig, axs = plt.subplots(nrows=5, ncols=2, figsize=(14, 10))
    
    axs[0,0].plot(t, qi, ls='--')
    axs[0,0].set_title('Heat input rate')

    axs[0,1].plot(t, prm[:, 0])
    axs[0,1].set_title('Meltwater inflow')

    axs[1,0].plot(t, syn_data['M'] + syn_data['M_err'], 'k+')
    axs[1,0].plot(t, syn_data['M'])
    axs[1,0].set_title('Lake mass')

    axs[1,1].plot(t, syn_data['T'] + syn_data['T_err'],'k+')
    axs[1,1].plot(t, syn_data['T'])
    axs[1,1].set_title('Lake temperature')
    
    axs[2,0].plot(t, prm[:,1])
    axs[2,0].set_title('Outflow')
    
    axs[2,1].plot(t, syn_data['z'] + syn_data['z_err'],'k+')
    axs[2,1].plot(t, syn_data['z'])
    axs[2,1].set_title('Lake level')
    
    axs[3,0].plot(t, syn_data['X'] + syn_data['X_err'], 'k+')
    axs[3,0].plot(t, syn_data['X'])
    axs[3,0].set_title('Mg++ total amount')

    axs[3,1].plot(t, prm[:, 3])
    axs[3,1].set_title('Evaporated steam')

    axs[4,0].plot(t, syn_data['v'] + syn_data['v_err'], 'k+')
    axs[4,0].plot(t, syn_data['v'])
    axs[4,0].set_title('Lake volume')
    
    axs[4,1].plot(t, syn_data['Mg'] + syn_data['Mg_err'], 'k+')
    axs[4,1].plot(t, syn_data['Mg'])
    axs[4,1].set_title('Mg++ concentration')
    



In [ ]:
df

In [ ]:
c = clemb.Clemb(None, None, None, None)

In [ ]:
c._df = df
c._dates = df.index

In [ ]:
rs = c.run_forward(nsamples=1000, new=True)

In [ ]:
rs['lh'].max()

In [ ]:
import matplotlib.colors as colors

days = mdates.DayLocator()  # every day
months = mdates.MonthLocator()
monthFmt = mdates.DateFormatter('%Y-%m-%d')
dayFmt = mdates.DateFormatter('%d')

datet = rs['dates']
data = c._df
t_data = data.index
nsteps = datet.size
nresample = 500

show_exp = True

mpl.rcParams['figure.subplot.hspace'] = 0.5
fig, axs = plt.subplots(nrows=5, ncols=2, figsize=(14, 10))

axs[0,0].plot(t_data, np.ones(t_data.size)*4.5, ls='--')
axs[0,0].set_title('Wind speed [m/s]')

axs[0,1].plot(t_data, data['X'], ls='--')

model_X = rs['model'].sel(obs='X')
axs[0,1].fill_between(t_data, data['X']-3*data['X_err'],
                      data['X']+3*data['X_err'], alpha=0.5)
if show_exp:
    axs[0,1].plot(datet, model_X.mean(axis=1), 'k-')
    axs[0,1].plot(datet, model_X.mean(axis=1)+3*model_X.std(axis=1), 'k--')
    axs[0,1].plot(datet, model_X.mean(axis=1)-3*model_X.std(axis=1), 'k--')
axs[0,1].set_title('Mg++ amount [kt]')

axs[1,0].plot(t_data, data['T'], ls='--')
axs[1,0].fill_between(t_data, data['T']-3*data['T_err'],
                      data['T']+3*data['T_err'], alpha=0.5)

model_T = rs['model'].sel(obs='T')
if show_exp:
    axs[1,0].plot(datet, model_T.mean(axis=1), 'k-')
    axs[1,0].plot(datet, model_T.mean(axis=1)+3*model_T.std(axis=1), 'k--')
    axs[1,0].plot(datet, model_T.mean(axis=1)-3*model_T.std(axis=1), 'k--')
axs[1,0].set_title('Lake temperature [$^{\circ}C$]')

axs[1,1].plot(t_data, data['M'], ls='--')
axs[1,1].fill_between(t_data, data['M']-3*data['M_err'],
                      data['M']+3*data['M_err'], alpha=0.5)
model_M = rs['model'].sel(obs='M')
if show_exp:
    axs[1,1].plot(datet, model_M.mean(axis=1), 'k-')
    axs[1,1].plot(datet, model_M.mean(axis=1)+3*model_M.std(axis=1), 'k--')
    axs[1,1].plot(datet, model_M.mean(axis=1)-3*model_M.std(axis=1), 'k--')
axs[1,1].set_title('Lake mass [kt]')

for k in range(nsteps):
    axs[2,0].scatter([datet[k].data]*nresample, rs['q_in'][k], s=2, c=rs['lh'][k],
                     cmap=plt.cm.get_cmap('RdBu_r'), alpha=0.3,
                     norm=colors.Normalize(vmin=rs['lh'].min(), vmax=rs['lh'].max()))
exp_q_in = rs['exp'].loc[:,'q_in']
var_q_in = rs['var'].loc[:,'q_in']
if show_exp:
    axs[2,0].plot(datet, exp_q_in, 'k')
    axs[2,0].plot(datet, exp_q_in - 3*np.sqrt(var_q_in), 'k--')
    axs[2,0].plot(datet, exp_q_in + 3*np.sqrt(var_q_in), 'k--')
axs[2,0].plot(datet, qi[:-1], 'b')
axs[2,0].set_title('Heat input rate [MW]')

for k in range(nsteps):
    axs[2,1].scatter([datet[k].data]*nresample, rs['h'][k], s=2, c=rs['lh'][k],
                cmap=plt.cm.get_cmap('RdBu_r'), alpha=0.3)
exp_h = rs['exp'].loc[:,'h']
axs[2,1].plot(datet, exp_h, 'k')
axs[2,1].set_title('Enthalpy [TJ/kt]')

for k in range(nsteps):
    axs[3,0].scatter([datet[k].data]*nresample, rs['m_in'][k], s=2, c=rs['lh'][k],
                cmap=plt.cm.get_cmap('RdBu_r'), alpha=0.3)
exp_m_in = rs['exp'].loc[:,'m_in']
var_m_in = rs['var'].loc[:,'m_in']
if show_exp:
    axs[3,0].plot(datet, exp_m_in, 'k')
    axs[3,0].plot(datet, exp_m_in - 3*np.sqrt(var_m_in), 'k--')
    axs[3,0].plot(datet, exp_m_in + 3*np.sqrt(var_m_in), 'k--')
axs[3,0].plot(datet, prm[:-1,0], 'b')
axs[3,0].set_title('Inflow [kt/day]')

for k in range(nsteps):
    axs[3,1].scatter([datet[k].data]*nresample, rs['m_out'][k], s=2, c=rs['lh'][k],
                cmap=plt.cm.get_cmap('RdBu_r'), alpha=0.3)
exp_m_out = rs['exp'].loc[:,'m_out']
var_m_out = rs['var'].loc[:,'m_out']
if show_exp:
    axs[3,1].plot(datet, exp_m_out, 'k')
    axs[3,1].plot(datet, exp_m_out - 3*np.sqrt(var_m_out), 'k--')
    axs[3,1].plot(datet, exp_m_out + 3*np.sqrt(var_m_out), 'k--')
axs[3,1].plot(datet, prm[:-1,1], 'b')
axs[3,1].set_title('Outflow [kt/day]')

for k in range(nsteps):
    axs[4,0].scatter([datet[k].data]*nresample, rs['steam'][k], s=2, c=rs['lh'][k],
                     cmap=plt.cm.get_cmap('RdBu_r'), alpha=0.3)
axs[4,0].plot(datet, qi[:-1]*0.0864/H, 'b')
axs[4,0].set_title('Steam input [kt/day]')
    
for k in range(nsteps):
    axs[4,1].scatter([datet[k].data]*nresample, rs['mevap'][k], s=2, c=rs['lh'][k],
                     cmap=plt.cm.get_cmap('RdBu_r'), alpha=0.3)
_ = axs[4,1].set_title('Evaporation mass loss [kt/day]')

for row in range(5):
    for col in range(2):
        axs[row,col].xaxis.set_major_locator(months)
        axs[row,col].xaxis.set_major_formatter(monthFmt)
        axs[row,col].xaxis.set_minor_locator(days)
        axs[row,col].set_xlim(datet[0].data, datet[-1].data)


### Equations from Hurst et al., 1991
Energy balance for the lake:

$$
\frac{d}{dt}Q = Q_i - Q_e - cTM_o + cT_sM_s
$$

$Q$ = total energy of the lake

$T$ = temperature at any time $t$

$c$ = specific heat of lake water

$Q_i$ = heat input at the base of the lake

$Q_e$ = heat loss due to evaporation, radiation, solar heating (gain)

$M_o$ = total rate of outflow

$M_s$ = inflow rate from melt

$T_s$ = temperature of inflow

Assuming $T_s = 0 ^{\circ}C$:

$$
\begin{aligned}
\frac{d}{dt}Q & = Q_i - Q_e - cTM_o \\
              & = \frac{d}{dt}[cMT] \\
              & = cM\frac{dT}{dt} + cT\frac{dM}{dt}
\end{aligned}
$$

$$
\Rightarrow \qquad \frac{dT}{dt} = \frac{1}{cM}\left(Q_i - Q_e - cTM_o -cT\frac{dM}{dt}\right) 
$$

$M$ = mass of the water in the lake at time t

Mass balance:

$$
\frac{dM}{dt} = M_i + M_s - M_o - M_e
$$

$M_i$ = rate at which water or steam is added through the volcanic vent

$M_e$ = rate of evaporation losses

Ion concentration balance:

$$
\begin{aligned}
\frac{d}{dt}[MZ] & = Z_i M_i - Z M_o\\
\frac{dM}{dt} + M_o & = \frac{1}{Z} ( Z_i M_i - M \frac{dZ}{dt} ) 
\end{aligned}
$$

$Z$ = ion concentration in the lake

$Z_iM_i$ = rate of addition of ions through the lake bottom

If $Z_iM_i = 0$:

$$
\frac{dM}{dt} = -M_o - \frac{M}{Z}\frac{dZ}{dt}
$$

### Equations from code

Lake mass:

$$
\begin{aligned}
\rho & = 1.003 - 0.00033T \\
V & = f(h)\\
M & = V * \rho
\end{aligned}
$$
$\rho$ = density of lake water

$V$ = lake volume

$h$ = lake level

Outflow:

$$
M_o = M_n - M_{n-1} + M_{n-1} \left (0.98\frac{M_{n-1}Z_{n-1}}{M_nZ_n} -1.0 \right )
$$

Long wave radiation loss:

$$
\dot{E}_{long} = 5.67e^{-8}A[0.97(T - 1 + 273.15)^4 -  0.8(0.9 + 273.15)^4]
$$

$A$ = lake surface area

Free and forced convection:

$$
\begin{aligned}
\dot{E}_{evap} & = \\
&  A \sqrt{\left [ 2.2 (T - 1 - 0.9)^{\frac{1}{3}} (6.112e^{\frac{17.62(T-1)}{243.12(T-1)}} - 6.5) \right ] ^2+ \left [ \left ( 4.07e^{-3} \frac{w^{0.8}}{500^{0.2}} - \frac{1.107e^{-2}}{500} \right ) \left ( \frac{1}{800} ( 6.112e^{\frac{17.62(T-1)}{243.12(T-1)}} - 6.5) \right ) 2400000 \right ]^2}
\end{aligned}
$$

$w$ = wind speed

Solar heating:

$$
\dot{E}_{solar} = \Delta t A 1.5e^{-5} \left [ 1 + 0.5 cos \left (\frac{((m-1)3.14)}{6.0} \right ) \right ]
$$

$m$ = month of the year

Heat loss due to evaporation radiation and solar heating 

$$
Q_e = \dot{E}_{long} + \dot{E}_{evap} \left ( 1+0.948 \frac{1005}{2.4e^6} \frac{T - 1 - 0.9}{6.335e^{-3} + 6.718e^{-4}(T-1) -2.0887e^{-5}(T-1)^2 + 7.3095e^{-7}(T-1)^3 - 2.2e^{-3}} \right ) - \dot{E}_{solar}
$$

$$
M_e = \dot{E}_{evap}2.4e^{-12}
$$

In [ ]:
(datetime[1]-datetime[0]).days

In [ ]:
pd.date_range?